# Co-plotting CATMAID skeleton with flywire dataset

### This section contains example of fetching skeleton from a public catmaid instance and co-plotting with a flywire instance

### Import neccesary library modules now

In [1]:
import navis
import fafbseg
import pymaid

In [2]:
import pandas as pd
import numpy as np
import os
from copy import deepcopy

In [3]:
from pyroglancer.localserver import startdataserver, closedataserver
from pyroglancer.flywire import flywireurl2dict, add_flywirelayer, set_flywireviewerstate
from pyroglancer.ngviewer import openviewer

### Set configurations to fetch from data from CATMAID

In [4]:
publicurl = 'https://fafb.catmaid.virtualflybrain.org/'

In [5]:
working_rm = pymaid.CatmaidInstance(publicurl, api_token=None, project_id = 1)

INFO  : Global CATMAID instance set. Caching is ON. (pymaid)
INFO - 2021-05-19 23:30:41,498 - client - Global CATMAID instance set. Caching is ON.


### Get sample skids and neuropil meshes from CATMAID

In [6]:
catmiad_neuronlist=pymaid.get_neuron('/DA1 lPN',remote_instance = working_rm) #get some DA1 PNs..

WARNING - 2021-05-19 23:30:42,839 - connectionpool - Connection pool is full, discarding connection: fafb.catmaid.virtualflybrain.org
WARNING - 2021-05-19 23:30:42,887 - connectionpool - Connection pool is full, discarding connection: fafb.catmaid.virtualflybrain.org
WARNING - 2021-05-19 23:30:43,086 - connectionpool - Connection pool is full, discarding connection: fafb.catmaid.virtualflybrain.org
WARNING - 2021-05-19 23:30:43,132 - connectionpool - Connection pool is full, discarding connection: fafb.catmaid.virtualflybrain.org
WARNING - 2021-05-19 23:30:43,178 - connectionpool - Connection pool is full, discarding connection: fafb.catmaid.virtualflybrain.org


INFO - 2021-05-19 23:30:43,574 - utils - NumExpr defaulting to 8 threads.


In [7]:
vols = pymaid.get_volume(['v14.neuropil', 'AL_R'], color=(255, 0, 0, .2))

### Transform them to flywire space

In [8]:
flywire_neuron=navis.xform_brain(catmiad_neuronlist,source='FAFB14', target='FLYWIRE')

Transform path: FAFB14 -> FAFB14raw -> FLYWIREraw -> FLYWIRE


In [9]:
flywirevol = {}
flywirevol['v14.neuropil']=navis.xform_brain(vols['v14.neuropil'],source='FAFB14', target='FLYWIRE')
flywirevol['AL_R']=navis.xform_brain(vols['AL_R'],source='FAFB14', target='FLYWIRE')

Transform path: FAFB14 -> FAFB14raw -> FLYWIREraw -> FLYWIRE
Transform path: FAFB14 -> FAFB14raw -> FLYWIREraw -> FLYWIRE


In [10]:
flywirevol['v14.neuropil'].id = 200
flywirevol['AL_R'].id = 300
flywirevol

{'v14.neuropil': <navis.Volume(name=None, id=200, color=(0.85, 0.85, 0.85, 0.2), vertices.shape=(8997, 3), faces.shape=(18000, 3))>,
 'AL_R': <navis.Volume(name=None, id=300, color=(0.85, 0.85, 0.85, 0.2), vertices.shape=(622, 3), faces.shape=(1240, 3))>}

### Start the dataserver to host precomputed data

In [11]:
startdataserver()

Serving data from:  /var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmp_5o756gr


### Get the url from flywire to append data onto

In [12]:
shorturl = 'https://ngl.flywire.ai/?json_url=https://globalv1.flywire-daf.com/nglstate/5644175227748352'
#This flywire segments has some stuff similar to the DA1 PNs, you will notice their closeness in the final plot

Serving directory at http://127.0.0.1:8000


### Add skids to flywire layers

In [13]:
tmpviewer = add_flywirelayer(flywireurl2dict(shorturl), layer_kws = {'type': 'skeletons',
                                                                     'source': flywire_neuron[7:9],
                                                                     'name': 'catmaid_skels',
                                                                     'color': 'red'}) 
#'alpha': 0.3, doesn't work yet in skeleton layers..

/private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmp_5o756gr/precomputed/catmaid_skels/skeletons/57323
/private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmp_5o756gr/precomputed/catmaid_skels/skeletons/57353
creating: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmp_5o756gr/precomputed/catmaid_skels/skeletons/seg_props
{'57323': ['#ff0000'], '57353': ['#ff0000']}
flywire url at: https://ngl.flywire.ai/?json_url=https://globalv1.flywire-daf.com/nglstate/5174672299655168


### Add neuropil meshes to flywire layers

In [14]:
shorturl = tmpviewer

In [15]:
tmpviewer = add_flywirelayer(flywireurl2dict(shorturl), layer_kws = {'type': 'volumes',
                                                                     'source': [flywirevol['v14.neuropil'],
                                                                                flywirevol['AL_R']],
                                                                     'name': 'catmaid_neuropils',
                                                                     'color': ['grey', 'magenta'], 
                                                                     'alpha': 0.3})

mesh/200
Seg id is: 200
Full filepath: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmp_5o756gr/precomputed/catmaid_neuropils/mesh/200
mesh/300
Seg id is: 300
Full filepath: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmp_5o756gr/precomputed/catmaid_neuropils/mesh/300
creating: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmp_5o756gr/precomputed/catmaid_neuropils/mesh/segment_properties
creating: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmp_5o756gr/precomputed/catmaid_neuropils/mesh/segment_names
flywire url at: https://ngl.flywire.ai/?json_url=https://globalv1.flywire-daf.com/nglstate/6708937697001472


In [16]:
shorturl = tmpviewer

In [17]:
layout = {'type':'xy-3d','orthographicProjection':'True'}

In [18]:
tmpviewer,shorturl = set_flywireviewerstate(shorturl, axis_lines=False, bounding_box=False, layout = layout)

flywire url at: https://ngl.flywire.ai/?json_url=https://globalv1.flywire-daf.com/nglstate/4984286184734720


### Screenshot of the flywire instance

#### The grey mesh is the v14.1 neuropil, magenta is the Antenna lobe (Right), you can see the skeletons from CATMAID (red) plotted in the skeleton layer, and some similar neurons (meshes) of the flywire dataset plotted in green and yellow

![alt text](pics/catmaid2flywire.png "catmaid2flywire")

### Add synapses and other annotations to flywire layers (not yet working)

In [ ]:
ngdict = flywireurl2dict(shorturl)

In [ ]:
tmpviewer = add_flywirelayer(ngdict, layer_kws = {'type': 'synapses','source': flywire_neuron,
                                                 'ngspace': 'FAFBv14.1'})

In [ ]:
temp_pts = pd.DataFrame([[123072, 47001, 3375]],columns=['x','y','z'])
temp_pts = pd.DataFrame([[123072, 47001, 3375], [120000, 17001, 3000]], columns=['x', 'y', 'z'])
temp_pts['description'] = ['center_pt','above_pt']

In [ ]:
#plot landmarks..
tmpviewer = add_flywirelayer(ngdict, layer_kws = {'type': 'points','name': 'landmarks',
                                                  'ngspace': 'FAFBv14.1',
                                                  'source': temp_pts,'color': 'red'})

In [ ]:
tmpviewer

### Close the dataserver

In [19]:
closedataserver()

Closing server at http://127.0.0.1:8000
Cleaning directory at /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmp_5o756gr
